# (Optional) 0. TrOCR 훈련 텍스트 데이터 생성

--- 

TrOCR 훈련 데이터를 만들기 위한 데이터 가공 과정입니다. 이미 ocr_dataset_poc.csv로 저장되어 있기 때문에 이 모듈을 실습하실 필요는 없지만, 데이터셋 가공 과정을 파악하거나 이에 대한 영감을 얻고 싶으신 분들은 코드를 한 줄씩 수행해 보세요. 

In [ ]:
!pip install -r requirements.txt

In [ ]:
import re
import random
import pandas as pd
import multiprocessing
from tqdm import tqdm
from datasets import load_dataset, load_metric
from IPython.display import display, HTML
from collections.abc import Iterable
from joblib import Parallel, delayed
from kiwipiepy import Kiwi
kiwi = Kiwi()
num_cores = multiprocessing.cpu_count()

<br>

## 뉴스 데이터셋 가공
---
저자가 가공한 뉴스 데이터셋을 가공하여 훈련 데이터를 생성합니다. 데이터셋의 샘플 개수는 약 2만여건에 불과하지만, 각 레코드를 문장 분리하여 신규 데이터셋을 생성하면 샘플 개수가 증가합니다.
- 데이터셋 출처: https://huggingface.co/datasets/daekeun-ml/naver-news-summarization-ko

In [ ]:
news_datasets = load_dataset('daekeun-ml/naver-news-summarization-ko')
news_df = pd.DataFrame(news_datasets["train"]['document'], columns=['document'])

In [ ]:
def split_sentences(datasets,idx):
    document = datasets[idx]['document']
    splits = kiwi.split_into_sents(document, return_tokens=False)
    return [s.text for s in splits]    

def flatten(lis):
    for item in lis:
        if isinstance(item, Iterable) and not isinstance(item, str):
            for x in flatten(item):
                yield x
        else:        
             yield item

### 문장 분리

Kiwi 파이썬 래퍼 (https://github.com/bab2min/kiwipiepy) 를 사용하여 문장을 분리합니다. 문장 분리에 많은 시간이 소요되는데, 병렬 처리를 통해 처리 시간을 단축할 수 있습니다.

In [ ]:
num_samples = len(news_df)
#num_samples = 200
out = Parallel(n_jobs=num_cores, backend='threading')(
    delayed(split_sentences)(datasets=news_datasets['train'],idx=idx) for idx in tqdm(range(0, num_samples), miniters=1000)
)

In [ ]:
def preprocessing_news(df):
    import re
    
    # Remove punctuations
    df["document"] = df["document"].apply(lambda s: ' '.join(re.sub("[.,!?:;-=...@#_]", " ", str(s)).split()))
    df["document"] = df["document"].apply(lambda s: ' '.join(re.sub("[ᅳㅡ]", "", str(s)).split()))
    df["document"] = df["document"].apply(lambda s: ' '.join(re.sub("[--]", "", str(s)).split()))
    df["document"] = df["document"].apply(lambda s: ' '.join(re.sub("[\.\,\(\)\{\}\[\]\`\'\!\?\:\;\-\=]", " ", str(s)).split()))
    df["document"] = df["document"].apply(lambda s: ' '.join(re.sub("[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]", "", s).split()))

    # Remove links
    df["document"] = df["document"].apply(lambda s: ' '.join(re.sub("(w+://S+)", " ", s).split()))
        
    return df

In [ ]:
news_texts = list(flatten(out))
news_df = pd.DataFrame(news_texts, columns=['document'])
news_df = preprocessing_news(news_df)
# news_df["document"] = news_df["document"].apply(lambda s: ' '.join(re.sub("[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]", "", s).split()))
# news_df["document"] = news_df["document"].apply(lambda s: ' '.join(re.sub("(w+://S+)", " ", s).split()))

In [ ]:
news_df.head()

<br>

## 네이버 영화 리뷰 데이터셋 가공 
---

네이버 영화 리뷰 데이터셋을 가공합니다. 더 많은 훈련 데이터를 확보하기 위해, 각 샘플의 문장 길이가 일정 이상일 때 문장 분리를 수행합니다.
- 데이터셋 출처: https://github.com/e9t/nsmc

In [ ]:
!curl -O https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
!curl -O https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt

In [ ]:
import pandas as pda
import numpy as np

train_df = pd.read_csv('ratings_train.txt', header=0, delimiter='\t')
test_df = pd.read_csv('ratings_test.txt', header=0, delimiter='\t')
df = pd.concat([train_df, test_df], ignore_index=True)

In [ ]:
def preprocessing_nsmc(df):
    import re
    
    # Remove consonant & vowel for Korean language
    df["clean_document"] = df["document"].apply(lambda s: ' '.join(re.sub("([ㄱ-ㅎㅏ-ㅣ]+)", "", str(s)).split()))
    
    # Remove punctuations
    df["clean_document"] = df["clean_document"].apply(lambda s: ' '.join(re.sub("[.,!?:;-=...@#_]", " ", str(s)).split()))
    df["clean_document"] = df["clean_document"].apply(lambda s: ' '.join(re.sub("[ᅳㅡ]", "", str(s)).split()))
    df["clean_document"] = df["clean_document"].apply(lambda s: ' '.join(re.sub("[--]", "", str(s)).split()))
    df["clean_document"] = df["clean_document"].apply(lambda s: ' '.join(re.sub("[\.\,\(\)\{\}\[\]\`\'\!\?\:\;\-\=]", " ", str(s)).split()))
    df["clean_document"] = df["clean_document"].apply(lambda s: ' '.join(re.sub("[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]", "", s).split()))

    # Remove non-korean characters
    df["clean_document"] = df["clean_document"].apply(lambda s: ' '.join(re.sub("[^가-힣ㄱ-하-ㅣ\\s]", "", str(s)).split()))

    # Remove links
    df["clean_document"] = df["clean_document"].apply(lambda s: ' '.join(re.sub("(w+://S+)", " ", s).split()))
        
    return df

df = preprocessing_nsmc(df)

In [ ]:
min_chars = 5
max_chars = 32
nsmc_short_df = df[(df["clean_document"].str.len() >= min_chars) & (df["clean_document"].str.len() < max_chars)]
nsmc_long_df = df[df["clean_document"].str.len() >= max_chars]

In [ ]:
def split_sentences(df,idx):
    document = df['document'].iloc[idx]
    splits = kiwi.split_into_sents(document, return_tokens=False)
    return [s.text for s in splits]    

num_samples = len(nsmc_long_df)
out = Parallel(n_jobs=num_cores, backend='threading')(
    delayed(split_sentences)(df=nsmc_long_df,idx=idx) for idx in tqdm(range(0, num_samples), miniters=10000)
)

In [ ]:
nsmc_texts = list(flatten(out))
nsmc_long_df = pd.DataFrame(nsmc_texts, columns=['document'])
nsmc_long_df = preprocessing_nsmc(nsmc_long_df)

In [ ]:
nsmc_short_df = nsmc_short_df["clean_document"].to_frame(name="document")
nsmc_long_df = nsmc_long_df["clean_document"].to_frame(name="document")

<br>

## 챗봇 데이터셋 가공
---

챗봇 데이터셋을 가공합니다.
- 데이터셋 출처: https://github.com/songys/Chatbot_data

In [ ]:
import urllib
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", 
                           filename="chatbot_train.csv")
chatbot_df = pd.read_csv('chatbot_train.csv')
chatbot_df.head()

질문 문장과 응답 문장을 분리하여 개별 데이터프레임을 생성합니다.

In [ ]:
chatbot_q_df = chatbot_df['Q'].to_frame()
chatbot_q_df.columns = ['document']
chatbot_q_df = chatbot_q_df.drop_duplicates()

chatbot_a_df = chatbot_df['A'].to_frame()
chatbot_a_df.columns = ['document']
chatbot_a_df = chatbot_a_df.drop_duplicates()

In [ ]:
def preprocessing_chatbot(df, min_chars=4):
    df["document"] = df["document"].apply(lambda s: ' '.join(re.sub("[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]", "", s).split()))
    # Remove rows if text has less than min characters
    df = df[df["document"].str.len() >= min_chars]

    return df

chatbot_q_df = preprocessing_chatbot(chatbot_q_df)
chatbot_a_df = preprocessing_chatbot(chatbot_a_df)

<br>

## 최종 데이터셋 취합
---

In [ ]:
nsmc_short_df['category'] = 'nsmc'
nsmc_long_df['category'] = 'nsmc'
news_df['category'] = 'news'
chatbot_q_df['category'] = 'chatbot'
chatbot_a_df['category'] = 'chatbot'

final_df = pd.concat(
    [nsmc_short_df, nsmc_long_df, news_df, chatbot_q_df, chatbot_a_df], 
    ignore_index=True
)
final_df = final_df[final_df["document"].str.len() >= 5]
final_df['document'] = final_df['document'].str.strip()

In [ ]:
final_df['category'].value_counts()

In [ ]:
final_df.to_csv('ocr_dataset_poc.csv', index=False)

<br>

## Clean up
---

In [ ]:
!rm ratings_train.txt ratings_test.txt chatbot_train.csv